In [1]:
# %load mosquitoes_train.py
import json
import skimage
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

import coco
import utils
import model as modellib
#import visualize
from model import log
dataset_dir = './data'
annotations = "via_region_data.json"

class MosquitoesConfig(coco.Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "mosquitoes"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Uncomment to train on 8 GPUs (default is 1)
    #GPU_COUNT = 2

    # Number of classes (including background)
    # NUM_CLASSES = 1 + 80  # COCO has 80 classes
    NUM_CLASSES = 1 + 2  # Person and background

    NUM_KEYPOINTS = 3
    MINI_MASK_SHAPE = [256, 256]
    MASK_SHAPE = [28, 28]
    KEYPOINT_MASK_SHAPE = [56,56]
    # DETECTION_MAX_INSTANCES = 50
    TRAIN_ROIS_PER_IMAGE = 50
    MAX_GT_INSTANCES = 128
    RPN_TRAIN_ANCHORS_PER_IMAGE = 150
    USE_MINI_MASK = True
    MASK_POOL_SIZE = 14
    KEYPOINT_MASK_POOL_SIZE = 7
    LEARNING_RATE = 0.002
    STEPS_PER_EPOCH = 1000
    WEIGHT_LOSS = True
    KEYPOINT_THRESHOLD = 0.005

config = MosquitoesConfig()
config.display()


class MosquitoesDataset(utils.Dataset):
    def __init__(self):
        super().__init__(MosquitoesDataset)
        num_classes = 3
        self.task_type = "person_keypoints"
        # the connection between 2 close keypoints
        self._skeleton = []
        # keypoint names
        # ["prob","head","tail"]
        self._keypoint_names = []

    def load_dataset(self, dataset_dir, subset='train'):
        """Load a subset of the Balloon dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
#         self.add_class("balloon", 1, "balloon")

#         # Train or validation dataset?
        assert subset in ["train", "val"]
#         dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        if subset=='train':
             annotations = json.load(open(os.path.join(dataset_dir, "annotations","train.json")))
        else:
             annotations = json.load(open(os.path.join(dataset_dir, "annotations","val.json")))

        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        #remove wrong annotations
        remove_index = 0
        for index, a in enumerate(annotations):
            if a['filename']=='8_0083_4.jpg':
                remove_index = index
        annotations.pop(remove_index)

        if subset=='train':
            annotations = annotations[:360]
        else:
            annotations = annotations[360:]
        print(subset,' num_images: ',len(annotations))
        # Add images
        for a in annotations:
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.

            image_path = os.path.join(dataset_dir, "images", a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
            num_mosquitoes = 0
            cl = list()
            bb = list()
            kp = list()
            for index, attr in enumerate(a['regions']):
                #print(index,attr)
                if (index%5==0):
                    #attr['region_attributes'].setdefault('class','1')# if the 'class' is missing, fill out automatically, but this error seldom happens
                    if not 'class' in attr['region_attributes']:
                        attr['region_attributes'].setdefault('class','1')# if the 'class' is missing, fill out automatically, but this error seldom happens
                    if not attr['region_attributes']['class']:#if the va;ues of 'class' is missing, fill out.
                        cl.append(str(random.randint(1,2)))
                    if attr['region_attributes']['class']:
                        cl.append(int(attr['region_attributes']['class']))
                    if 'y' in attr['shape_attributes']:# eror: if a extral point is marked without sense
                        bb.append([attr['shape_attributes']['y'], attr['shape_attributes']['x'], attr['shape_attributes']['height'], attr['shape_attributes']['width']])

                elif (index%5==1):
                    pass
                else:
                    kp.append(( attr['shape_attributes']['cy'], attr['shape_attributes']['cx']))

                num_mosquitoes += 1

            num_mosquitoes = int(num_mosquitoes/5)

            self.add_image(
                "mosquitoes",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                cl = cl,
                bounding_box = bb,
                key_points = kp,
                num_mosquitoes = num_mosquitoes
                )

    def load_bbox(self, image_id):
        bounding_box = self.image_info[image_id]['bounding_box']
        num_mosquitoes = self.image_info[image_id]['num_mosquitoes']
        bounding_box = np.reshape(bounding_box, (-1,4))
        bounding_box[:,2] += bounding_box[:,0]
        bounding_box[:,3] += bounding_box[:,1]
        return bounding_box

    def load_image(self, image_id):
        image_path = self.image_info[image_id]['path']
        image = skimage.io.imread(image_path)
        return image

    def load_keypoints(self, image_id):
        """Load person keypoints for the given image.

        Returns:
        key_points: num_keypoints coordinates and visibility (x,y,v)  [num_person,num_keypoints,3] of num_person
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks, here is always equal to [num_person, 1]
        """
        # If not a COCO image, delegate to parent class.
#         print(self.image_info)
#         image_info = self.image_info[image_id]
#         if image_info["source"] != "coco":
#             return super(CocoDataset, self).load_mask(image_id)

        keypoints = []
        class_ids = []
        instance_masks = []
        info = self.image_info[image_id]
        num_mosquitoes = info['num_mosquitoes']
        # Build mask of shape [height, width, instance_count] and list
        # of class IDs that correspond to each channel of the mask.

        for index in range(0, int(info['num_mosquitoes'])):
            class_id = info['cl'][index]

            m = np.zeros((info['height'], info['width']), dtype=np.uint8)
            # generate masks
            for m_index in range(0,3):
                #m_index = index*3 + m_index
                #m = np.zeros((info['height'], info['width']), dtype=np.uint8)
                x = info['key_points'][m_index][0]
                y = info['key_points'][m_index][1]
                m[x,y] = 255
            instance_masks.append(m)
            #load keypoints
            keypoints = info["key_points"]
            keypoints = np.reshape(keypoints,(-1,2))
            new_col = np.ones((keypoints.shape[0],1))+1
            keypoints = np.hstack((keypoints, new_col))
            keypoints = np.reshape(keypoints, (num_mosquitoes,3,3))
#             keypoints.append(keypoint)
            class_ids.append(class_id)
        # Pack instance masks into an array
#         if class_ids:
        keypoints = np.array(keypoints,dtype=np.int32)
        class_ids = np.array(class_ids, dtype=np.int32)
        masks = np.stack(instance_masks, axis=2)

        return keypoints, masks, class_ids
#         else:
#             # Call super class to return an empty mask
#             return super(CocoDataset, self).load_keypoints(image_id)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        return info["path"]


if __name__=='__main__':

    train_dataset_keypoints = MosquitoesDataset()
    train_dataset_keypoints.load_dataset(dataset_dir, "train")
    train_dataset_keypoints.prepare()

    val_dataset_keypoints = MosquitoesDataset()
    val_dataset_keypoints.load_dataset(dataset_dir, "val")
    val_dataset_keypoints.prepare()

    ROOT_DIR = os.getcwd()
    MODEL_DIR = os.path.join(ROOT_DIR, "logs")
    config = MosquitoesConfig()
# Local path to trained weights file
    COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Create model object in inference mode.
    model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
    model.load_weights(COCO_MODEL_PATH, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
    print("Loading weights from ", COCO_MODEL_PATH)

# Training - Stage 1
    print("Train heads")
    model.train(train_dataset_keypoints, val_dataset_keypoints,\
            learning_rate=config.LEARNING_RATE,\
            epochs=15,\
            layers="heads")

# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
    print("Training Resnet layer 4+")
    model.train(train_dataset_keypoints, val_dataset_keypoints,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=20,
            layers='4+')
# Training - Stage 3
# Finetune layers from ResNet stage 3 and up
    print("Training Resnet layer 3+")
    model.train(train_dataset_keypoints, val_dataset_keypoints,
            learning_rate=config.LEARNING_RATE / 100,
            epochs=20,
            layers='all')

/home/tda1/anaconda3/envs/tensorflow_tf_19_keras216/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/tda1/anaconda3/envs/tensorflow_tf_19_keras216/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/tda1/anaconda3/envs/tensorflow_tf_19_keras216/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.



Configurations Superlee:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
KEYPOINT_MASK_POOL_SIZE        7
KEYPOINT_MASK_SHAPE            [56, 56]
KEYPOINT_THRESHOLD             0.005
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.002
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               128
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                [256, 256]
NAME                           mosquitoes
NUM

Loading weights from  /home/tda1/jm1/Mask_RCNN_Humanpose/mask_rcnn_coco.h5
Train heads

Starting at epoch 0. LR=0.002

Checkpoint Path: /home/tda1/jm1/Mask_RCNN_Humanpose/logs/mosquitoes20180823T1056/mask_rcnn_mosquitoes_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_keypoint_mask_conv1   (TimeDistributed)
mrcnn_keypoint_mask_bn1   (TimeDistributed)
mrcnn_keypoint_mask_conv2   (TimeDistributed)
mrcnn_keypoint_mask_bn2   (TimeDistributed)
mrcnn_keypoint_mask_conv3   (TimeDistributed)
mrcnn_keypoint_mask_bn3   (TimeDistributed)
mrcnn_keypoint_mask_conv4   (TimeDistributed)
mrcnn_keypoint_mask_bn4   (TimeDistri

/home/tda1/anaconda3/envs/tensorflow_tf_19_keras216/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
1000/1000 [==============================] - 1016s 1s/step - loss: 6.1028 - rpn_class_loss: 0.0088 - rpn_bbox_loss: 0.2159 - mrcnn_class_loss: 0.4933 - mrcnn_bbox_loss: 0.2928 - keypoint_mrcnn_mask_loss: 5.0812 - mrcnn_mask_loss: 0.0108 - val_loss: 1.4337 - val_rpn_class_loss: 0.0421 - val_rpn_bbox_loss: 0.4220 - val_mrcnn_class_loss: 0.5973 - val_mrcnn_bbox_loss: 0.3705 - val_keypoint_mrcnn_mask_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0019
Epoch 2/15
1000/1000 [==============================] - 998s 998ms/step - loss: 5.6741 - rpn_class_loss: 0.0063 - rpn_bbox_loss: 0.1578 - mrcnn_class_loss: 0.4019 - mrcnn_bbox_loss: 0.1884 - keypoint_mrcnn_mask_loss: 4.9166 - mrcnn_mask_loss: 0.0032 - val_loss: 9.7643 - val_rpn_class_loss: 0.0811 - val_rpn_bbox_loss: 0.5312 - val_mrcnn_class_loss: 0.9294 - val_mrcnn_bbox_loss: 0.4673 - val_keypoint_mrcnn_mask_loss: 7.7527 - val_mrcnn_mask_loss: 0.0026
Epoch 3/15
1000/1000 [==============================] - 986s 986ms/step - loss: 5.3311

Epoch 16/20
1000/1000 [==============================] - 1410s 1s/step - loss: 2.6193 - rpn_class_loss: 0.0032 - rpn_bbox_loss: 0.0436 - mrcnn_class_loss: 0.0892 - mrcnn_bbox_loss: 0.0561 - keypoint_mrcnn_mask_loss: 2.4242 - mrcnn_mask_loss: 0.0030 - val_loss: 2.9422 - val_rpn_class_loss: 0.1238 - val_rpn_bbox_loss: 0.5412 - val_mrcnn_class_loss: 1.4605 - val_mrcnn_bbox_loss: 0.8160 - val_keypoint_mrcnn_mask_loss: 0.0000e+00 - val_mrcnn_mask_loss: 6.7362e-04
Epoch 17/20
1000/1000 [==============================] - 1341s 1s/step - loss: 2.5975 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.0365 - mrcnn_class_loss: 0.0857 - mrcnn_bbox_loss: 0.0523 - keypoint_mrcnn_mask_loss: 2.4172 - mrcnn_mask_loss: 0.0030 - val_loss: 3.6936 - val_rpn_class_loss: 0.1730 - val_rpn_bbox_loss: 0.6602 - val_mrcnn_class_loss: 2.0783 - val_mrcnn_bbox_loss: 0.7809 - val_keypoint_mrcnn_mask_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0012
Epoch 18/20
1000/1000 [==============================] - 1338s 1s/step - loss:

Training Resnet layer 4+

Starting at epoch 20. LR=0.0002

Checkpoint Path: /home/tda1/jm1/Mask_RCNN_Humanpose/logs/mosquitoes20180823T1056/mask_rcnn_mosquitoes_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_branch2c         (Conv2D)
bn4b_branch2c          (BatchNorm)
res4c_branch2a         (Conv2D)
bn4c_branch2a          (BatchNorm)
res4c_branch2b         (Conv2D)
bn4c_branch2b          (BatchNorm)
res4c_branch2c         (Conv2D)
bn4c_branch2c          (BatchNorm)
res4d_branch2a         (Conv2D)
bn4d_branch2a          (BatchNorm)
res4d_branch2b         (Conv2D)
bn4d_branch2b          (Batch

/home/tda1/anaconda3/envs/tensorflow_tf_19_keras216/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 21/30
1000/1000 [==============================] - 1426s 1s/step - loss: 2.3701 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.0270 - mrcnn_class_loss: 0.0668 - mrcnn_bbox_loss: 0.0384 - keypoint_mrcnn_mask_loss: 2.2326 - mrcnn_mask_loss: 0.0029 - val_loss: 3.8924 - val_rpn_class_loss: 0.1418 - val_rpn_bbox_loss: 0.5375 - val_mrcnn_class_loss: 2.4990 - val_mrcnn_bbox_loss: 0.7127 - val_keypoint_mrcnn_mask_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0013
Epoch 22/30
1000/1000 [==============================] - 1345s 1s/step - loss: 2.3197 - rpn_class_loss: 0.0023 - rpn_bbox_loss: 0.0257 - mrcnn_class_loss: 0.0682 - mrcnn_bbox_loss: 0.0410 - keypoint_mrcnn_mask_loss: 2.1797 - mrcnn_mask_loss: 0.0029 - val_loss: 3.4432 - val_rpn_class_loss: 0.1790 - val_rpn_bbox_loss: 0.5784 - val_mrcnn_class_loss: 1.9988 - val_mrcnn_bbox_loss: 0.6861 - val_keypoint_mrcnn_mask_loss: 0.0000e+00 - val_mrcnn_mask_loss: 8.9058e-04
Epoch 23/30
 527/1000 [==============>...............] - ETA: 10:35 - loss: 2.